In [ ]:
%matplotlib qt
from pathlib import Path
import json
import numpy as np
import joypy
import seaborn as sns

In [ ]:
subject = "bert"
subjects_dir = Path.cwd().parent / "data" / "subjects"
delays_dir = subjects_dir / subject / "delays"

In [ ]:
my_dict_1 = {}
for fname in delays_dir.iterdir():
    stem = fname.stem.lower()
    if stem.startswith("acq"):
        continue
    elif stem.startswith("artifact"):
        key = "ORICA"
    else:
        continue

    with open(fname, "r") as f:
        data = json.load(f)
    my_dict_1[key] = np.array(data) * 1e3

my_dict_2 = {}
for visit in range(1, 6):
    fname = delays_dir / f"method_delay_visit_{visit}.json"
    with open(fname, "r") as f:
        data = json.load(f)
    for key, val in data.items():
        my_dict_2[key] = np.array(val) * 1000

In [ ]:
joypy.joyplot(my_dict_1, fade=True, overlap=2, color="lightsteelblue")

In [ ]:
colors = sns.cubehelix_palette(n_colors=10, rot=-0.2, as_cmap=False)[4:6]
joypy.joyplot(my_dict_1, fade=True, overlap=2, color=colors)

In [ ]:
colors = sns.cubehelix_palette(n_colors=10, rot=-0.2, as_cmap=False)
joypy.joyplot(my_dict_2, fade=True, overlap=2, color=colors)

for example

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


fname = Path.cwd().parent / "data" / "sample" / "method_delays.csv"
df = pd.read_csv(fname, index_col=0)

## function for mapping text into plot
def label(x, color, label):
    ax = plt.gca()
    ax.text(0.8, .2, label, fontweight="bold", fontstyle='italic', color=color,
            ha="left", va="center", transform=ax.transAxes)
    
def plot_method_delays(df, method_names, xlim, ylim, bw_adjust=1, top=0.72):
    df_sub = df.query("method == @method_names")
    pal = sns.cubehelix_palette(len(method_names), rot=-.2, light=.7)

    g = sns.FacetGrid(
        df_sub, row="method", hue="method", aspect=14, height=.75,
        palette=pal, row_order=method_names, xlim=xlim, ylim=ylim
    )

    g.map(sns.kdeplot, "delay", bw_adjust=bw_adjust, clip_on=False, clip=xlim,
            fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, "delay", clip_on=False, color="w", clip=xlim,
            lw=2, bw_adjust=bw_adjust)
    g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
    g.map(label, "delay")

    g.figure.subplots_adjust(hspace=.15, top=top)
    g.set_titles("")
    g.set(yticks=[], ylabel="", xlabel=r"method delay ($m s$)")
    g.despine(bottom=True, left=True)
    return g

plot_method_delays(df, ["sensor_power", "individual_peak_power", "band_ratio"], xlim=[0, 0.2], ylim=[0, 50])
plot_method_delays(df, ["entropy", "argmax_freq", "sensor_connectivity_corr"], xlim=[0.2, 1.8], ylim=[0, 6], bw_adjust=1.5)
plot_method_delays(df, ["sensor_cfc", "sensor_connectivity_pli"], xlim=[-5, 30], ylim=[0, 0.2], bw_adjust=1.8, top=0.65)
plot_method_delays(df, ["sensor_graph_sqe", "sensor_graph_corr"], xlim=[30, 80], ylim=[0, .1])
plot_method_delays(df, ["source_connectivity_pli", "source_connectivity_corr", "source_power", "source_graph"], xlim=[0, 700], ylim=[0, 0.01], bw_adjust=1.5)